In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from dataloader.builder import build_dataset
from experiment_setup import build_estimator

from model.cnn import SimpleConv, Trainer

In [3]:
config = {
    'use_cuda': True,
    'batch_size': 128,
    'epochs': 50,
    'lr': 1e-2,
    'momentum': 0.5,
    'seed': 1,
    'log_interval': 10
}



In [4]:
# load data
mnist = build_dataset('mnist', val_size=10_000)
x_train, y_train = mnist.dataset('train')
x_val, y_val = mnist.dataset('val')

ood = build_dataset('fashion_mnist', val_size=0)
x_ood, _ = ood.dataset('train') 

In [5]:
# preprocess
x_train = x_train.reshape(-1, 1, 28, 28)
x_val = x_val.reshape(-1, 1, 28, 28)
x_ood = x_ood.reshape(-1, 1, 28, 28)
y_train = y_train.astype('long').reshape(-1)
y_val = y_val.astype('long').reshape(-1)
x_train /= 255.0
x_val /= 255.0

In [ ]:
train_samples = 500
model = SimpleConv()
trainer = Trainer(model)
trainer.fit(x_train[:train_samples], y_train[:train_samples], epochs=config['epochs'])
accuracy_score(y_val, trainer.predict(x_val))

In [ ]:
estimator = build_estimator('bald', trainer, num_classes=10)

In [ ]:
estimation_samples = train_samples
uq_train = estimator.estimate(x_train[:estimation_samples])
uq_val = estimator.estimate(x_val[:estimation_samples])
uq_ood = estimator.estimate(x_ood[:estimation_samples])

In [ ]:
df = pd.DataFrame()
df['uq'] = np.concatenate(
    (uq_train, uq_val, uq_ood))
df['dataset'] = ['train'] * len(uq_train) + ['val'] * len(uq_val) + ['ood']*len(uq_ood)

sns.boxplot(data=df, x='dataset',  y='uq')


In [ ]:
print('train')
print(pd.Series(uq_train).describe())
print()

print('val')
print(pd.Series(uq_val).describe())
print()

print('ood')
print(pd.Series(uq_ood).describe())
print()
